# Injecting dyslexia syle text

In this notebook we inject dyslexia style errors into WMT14 dataset and save it into a folder called "to_test"

In [1]:
import re
import pandas as pd
import copy
import numpy as np
import evaluate
from docx import Document
import os
import unittest
from datasets import load_dataset
class DataLoader:
    """
    Loader for benchmarking datasets to ensure universal formatting. To be used in conjunction with DyslexiaInjector.
    ...
    Attributes
    ----------
    path: str
        Path to csv, txt or docx file of the data. In the case of CSV there should only be 1 column
    data: list
        A list of striings
    dataset_name: str
        Name of the dataset that is used when saving the data
    ...
    Methods
    -------
    parse_txt(path)
        Parses a txt file and returns a list of strings
    fix_format(sentence)
        Fixes the formatting of a sentence
    save_as_txt(path)
        Saves the data as a txt file
    save_as_csv(path)
        Saves the data as a csv file
    save_as_docx(path)
        Saves the data as a docx file
    get_data()
        Returns the data
    create_deepcopy()
        Returns a deepcopy of the DataLoader instance
    get_name()
        Returns the dataset name
    get_number_of_sentences()
        Returns the number of sentences in the data
    get_number_of_words()
        Returns the number of words in the data
    get_number_of_letters()
        Returns the number of letters in the data
    edit_distance(reference_sentence, sentence)
        Returns the number of edits required to transform reference_sentence into sentence at word level
        edits include insertions, deletions and substitutions
        based on levenshtein distance
        also returns a dictionary of substitutions, insertions and deletions
    get_edit_distance(reference, manual_wer=False)
        Returns the number of edits required to transform data into reference at word level, substitutions, insertions and deletions the associated dictionaries
        and the WER (withouth alignment) if manual_wer is set to True
    get_individual_edit_distance(reference)
        Returns the number of edits required to transform data into reference at word level for each individual sentence
    combine_nested_dict(dict1, dict2)
        Combines two nested dictionaries
    combine_dicts(dict1, dict2)
        Combines two dictionaries
    get_bleue_score(reference)
        Returns bleu score of the data against a reference
    get_wer(reference)
        Returns the Word Error Rate (WER) of the data against a reference. With word alignment
    get_bert_score(reference)
        Returns the BERTScore similarity score of the data against a reference
    get_LaBSE(reference, model=None, tokenizer=None)
        Returns the LaBSE similarity score of the data against a reference which is a l2 norm between the reference and target sentences score.
        Score of 1 means the sentences are identical, closer to 0 means they are less similar semantically.
    ...

    Usage
    -------
    >>> from datasets import load_dataset
    >>> from DataLoader import DataLoader
    >>> dataset_wmt_enfr = load_dataset("wmt14",'fr-en', split='test')
    >>> to_translate = []
    >>> for i in range(len(dataset_wmt_enfr)):
    >>>     to_translate.append(dataset_wmt_enfr[i]['translation']['en'])
    >>> loader = DataLoader(data=to_translate, dataset_name="wmt14_enfr")
    >>> loader.save_as_txt("wmt14_enfr.txt")
    We can also use the text file to create a new DataLoader instance
    >>> loader2 = DataLoader(path="wmt14_enfr.txt", dataset_name="wmt14_enfr")
    """
    # Constructor
    def __init__(self, path=None, data=None, dataset_name=""):
        self.dataset_name = dataset_name
        if data is None and path is not None:
            #check path to see if file is txt or csv
            file_type = path.split(".")[-1]
            if file_type == "txt":
                self.data = self.parse_txt(path)
                self.data = [self.fix_format(sentence) for sentence in self.data]
            elif file_type == "csv":
                self.data = pd.read_csv(path, header=None)
                self.data = self.data[0].tolist()
                #fix any formatting issues
                self.data = [self.fix_format(sentence) for sentence in self.data]
            elif file_type == "docx":
                doc = Document(path)
                self.data = [self.fix_format(paragraph.text) for paragraph in doc.paragraphs]
            else:
                raise Exception("Invalid file type")
        elif data is not None:
            #check if data is a list or a df
            if isinstance(data, list):
                #format each sentence in data
                self.data = [self.fix_format(sentence) for sentence in data]
            else:
                raise Exception("Invalid data type, please pass in a list of sentences")
        else:
            raise Exception("Please pass in a path or data")

    def parse_txt(self, path):
        output = []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                output.append(self.fix_format(line))
        return output
                
    def fix_format(self, sentence):
        #remove spacing before punctuation
        sentence = re.sub(r'\s([?.!,"](?:\s|$))', r'\1', sentence)
        #replace any double spaces with single space
        sentence = re.sub(r'\s+', ' ', sentence)
        #remove any leading or trailing spaces
        sentence = sentence.strip()
        #make all quotes (german and french) english double quotes
        sentence = re.sub(r'«|»|„|“', '"', sentence)
        #make all single quotes english single quotes
        sentence = re.sub(r'‘|’', "'", sentence)
        #make all french guillemets english double quotes
        sentence = re.sub(r'‹|›', '"', sentence)
        #if sentence begins and ends with quotes and there are only two, remove them
        if sentence[0] == '"' and sentence[-1] == '"' and sentence.count('"') == 2:
            sentence = sentence[1:-1]
        elif sentence[0] == "'" and sentence[-1] == "'" and sentence.count("'") == 2:
            sentence = sentence[1:-1]
        return sentence

    def save_as_txt(self, path):
        with open(path, "w", encoding="utf-8") as f:
            for sentence in self.data:
                f.write(f"{sentence}\n")
        print(f"Saved {self.dataset_name} to {path}")
        return
    
    def save_as_csv(self, path):
        df = pd.DataFrame(self.data)
        df.to_csv(path, index=False, header=False, encoding='utf-8')
        print(f"Saved {self.dataset_name} to {path}")
        return
    
    def save_as_docx(self, path):
        document = Document()
        for sentence in self.data:
            document.add_paragraph(sentence)
        document.save(path)
        print(f"Saved {self.dataset_name} to {path}")
        return

    def get_data(self):
        return self.data

    def create_deepcopy(self):
        return DataLoader(data=copy.deepcopy(self.data), dataset_name=self.dataset_name)
        
    def get_name(self):
        return self.dataset_name

    def get_number_of_sentences(self):
        return len(self.data)
    
    def get_number_of_words(self):
        return sum([len(sentence.split()) for sentence in self.data])
    
    def get_number_of_letters(self):
        #need to ensure we only count letters and not punctuation
        return sum([len(re.sub(r'[^\w\s]','',sentence)) for sentence in self.data])

    def edit_distance(reference_sentence, sentence):
        """
        Returns the number of edits required to transform reference_sentence into sentence at word level
        edits include insertions, deletions and substitutions
        based on levenshtein distance
        also returns a dictionary of substitutions, insertions and deletions
        """
        substitutions = 0
        insertions = 0
        deletions = 0
        substitution_dict = {}
        insertion_dict = {}
        deletion_dict = {}
        #remove punctuation and split into words
        sentence = re.sub(r'[^\w\s]','',sentence).lower().split()
        reference_sentence = re.sub(r'[^\w\s]','',reference_sentence).lower().split()
        #create matrix
        matrix = np.zeros((len(reference_sentence)+1,len(sentence)+1))
        #fill in first row and column
        for i in range(len(reference_sentence)+1):
            matrix[i][0] = i
        for j in range(len(sentence)+1):
            matrix[0][j] = j
        #fill in rest of matrix
        for i in range(1,len(reference_sentence)+1):
            for j in range(1,len(sentence)+1):
                if sentence[j-1] == reference_sentence[i-1]:
                    matrix[i][j] = matrix[i-1][j-1]
                else:
                    matrix[i][j] = min(matrix[i-1][j-1], matrix[i-1][j], matrix[i][j-1])+1
        #backtrack to find edits
        i = len(reference_sentence)
        j = len(sentence)
        while i > 0 and j > 0:
            if sentence[j-1] == reference_sentence[i-1]:
                i -= 1
                j -= 1
            else:
                if matrix[i][j] == matrix[i-1][j-1]+1:
                    substitutions += 1
                    if reference_sentence[i-1] not in substitution_dict:
                        substitution_dict[reference_sentence[i-1]] = {sentence[j-1]:1}
                    else:
                        if sentence[j-1] not in substitution_dict[reference_sentence[i-1]]:
                            substitution_dict[reference_sentence[i-1]][sentence[j-1]] = 1
                        else:
                            substitution_dict[reference_sentence[i-1]][sentence[j-1]] += 1
                    i -= 1
                    j -= 1
                elif matrix[i][j] == matrix[i-1][j]+1:
                    deletions += 1
                    if reference_sentence[i-1] not in deletion_dict:
                        deletion_dict[reference_sentence[i-1]] = 1
                    else:
                        deletion_dict[reference_sentence[i-1]] += 1
                    i -= 1
                elif matrix[i][j] == matrix[i][j-1]+1:
                    insertions += 1
                    if sentence[j-1] not in insertion_dict:
                        insertion_dict[sentence[j-1]] = 1
                    else:
                        insertion_dict[sentence[j-1]] += 1
                    j -= 1
        while i > 0:
            deletions += 1
            if reference_sentence[i-1] not in deletion_dict:
                deletion_dict[reference_sentence[i-1]] = 1
            else:
                deletion_dict[reference_sentence[i-1]] += 1
            i -= 1
        while j > 0:
            insertions += 1
            if sentence[j-1] not in insertion_dict:
                insertion_dict[sentence[j-1]] = 1
            else:
                insertion_dict[sentence[j-1]] += 1
            j -= 1
        distance = substitutions+insertions+deletions
        return substitutions, insertions, deletions, substitution_dict, insertion_dict, deletion_dict, distance
        
    def get_edit_distance(self, reference, manual_wer=False):
        """
        Returns the number of edits required to transform data into reference at word level, substitutions, insertions and deletions the associated dictionaries
        and the WER (withouth alignment) if manual_wer is set to True
        """
        if type(reference) == list:
            substitutions = 0
            insertions = 0
            deletions = 0
            all_sub = {}
            all_ins = {}
            all_del = {}
            distance = 0
            for i in range(len(self.data)):
                sub, ins, dele, substitution_dict, insertion_dict, deletion_dict, dist = DataLoader.edit_distance(reference[i], self.data[i], )
                all_sub = self.combine_nested_dict(all_sub, substitution_dict)
                all_ins = self.combine_dicts(all_ins, insertion_dict)
                all_del = self.combine_dicts(all_del, deletion_dict)
                substitutions += sub
                insertions += ins
                deletions += dele
                distance += dist
            if manual_wer:
                return substitutions, insertions, deletions, all_sub, all_ins, all_del, distance, distance/(sum([len(sentence.split()) for sentence in reference]))
            return substitutions, insertions, deletions, all_sub, all_ins, all_del, distance
        elif type(reference) == DataLoader:
            return self.get_edit_distance(reference.get_data(), manual_wer=manual_wer)
        else:
            raise Exception("Invalid reference type, please pass in a list or DataLoader instance")

    def get_individual_edit_distance(self, reference):
        """
        Returns the number of edits required to transform data into reference at word level for each individual sentence
        """
        if type(reference) == list:
            output = []
            for i in range(len(self.data)):
                sub, ins, dele, substitution_dict, insertion_dict, deletion_dict, distance = DataLoader.edit_distance(reference[i], self.data[i], )
                output.append((sub, ins, dele, substitution_dict, insertion_dict, deletion_dict, distance))
            return output
        elif type(reference) == DataLoader:
            return self.get_individual_edit_distance(reference.get_data())
        else:
            raise Exception("Invalid reference type, please pass in a list or DataLoader instance")       

    def combine_nested_dict(self, dict1, dict2):
        for key in dict2:
            if key not in dict1:
                dict1[key] = dict2[key]
            else:
                for key2 in dict2[key]:
                    if key2 not in dict1[key]:
                        dict1[key][key2] = dict2[key][key2]
                    else:
                        dict1[key][key2] += dict2[key][key2]
        return dict1
    
    def combine_dicts(self, dict1, dict2):
        for key in dict2:
            if key not in dict1:
                dict1[key] = dict2[key]
            else:
                dict1[key] += dict2[key]
        return dict1

    def get_bleue_score(self, reference):
        #returns bleu score of the data against a reference
        bleu = evaluate.load("bleu")
        if type(reference) == list:
            return bleu.compute(predictions=self.data, references=reference)
        elif type(reference) == DataLoader:
            return bleu.compute(predictions=self.data, references=reference.get_data())
        else:
            raise Exception("Invalid reference type, please pass in a list or DataLoader instance")

    def get_wer(self, reference):
        """
        Returns the Word Error Rate (WER) of the data against a reference. With word alignment
        """
        wer = evaluate.load("wer")
        if type(reference) == list:
            return wer.compute(predictions=self.data, references=reference)
        elif type(reference) == DataLoader:
            return wer.compute(predictions=self.data, references=reference.get_data())
        else:
            raise Exception("Invalid reference type, please pass in a list or DataLoader instance")


    def get_bert_score(self, reference):
        """
        Returns the BERTScore similarity score of the data against a reference.
        """
        bert = evaluate.load("bertscore")
        if type(reference) == list:
            return bert.compute(predictions=self.data, references=reference, lang="fr")
        elif type(reference) == DataLoader:
            return bert.compute(predictions=self.data, references=reference.get_data(), lang="fr")
        else:
            raise Exception("Invalid reference type, please pass in a list or DataLoader instance")

    def get_LaBSE(self, reference, model=None, tokenizer=None):
        """
        Returns the LaBSE similarity score of the data against a reference which is a l2 norm between the reference and target sentences score.
        Score of 1 means the sentences are identical, closer to 0 means they are less similar semantically.
        """
        if model is None:
            model = BertModel.from_pretrained("setu4993/LaBSE")
        if tokenizer is None:
            tokenizer = BertTokenizerFast.from_pretrained("setu4993/LaBSE")
        if type(reference) == list:
            pass
        elif type(reference) == DataLoader:
            reference = reference.get_data()
        else:
            raise Exception("Invalid reference type, please pass in a list or DataLoader instance")
        target = self.data
        reference_inputs = tokenizer(reference, return_tensors="pt", padding=True).to("cuda")
        target_inputs = tokenizer(target, return_tensors="pt", padding=True).to("cuda")
        with torch.no_grad():
            reference_outputs = model(**reference_inputs)
            target_outputs = model(**target_inputs)
        reference_embeddings = reference_outputs.pooler_output
        target_embeddings = target_outputs.pooler_output
        return self.similarity(reference_embeddings, target_embeddings)

class DyslexiaInjector:
    """
    This class is used to inject dyslexia into a dataset. It can be used to inject homophones and confusing letters.
    ...
    Attributes
    ----------
    load : DataLoader
        The DataLoader object that contains the data that needs to be injected
    homophones_dict : dict
        A dictionary that contains the homophones
    confusing_letters_dict : dict
        A dictionary that contains the confusing letters
    Methods
    -------
    load_homophones(path)
        Loads the homophones from a pickle file
    load_confusing_letters(path)
        Loads the confusing letters from a pickle file
    injection_swap(p_start=0, p_end=1, step_size=0.1, save_path="", save_format="both")
        Injects dyslexia into the dataset by swapping words and letters.
    get_homophones(word)
        Returns the homophones of a word
    get_confusing_letters(letter)
        Returns the confusing letters of a letter
    injector(sentence, p_homophone, p_letter)
        Injects dyslexia into a sentence with a given probability
    Usage
    -------
    >>> from datasets import load_dataset
    >>> from DataLoader import DataLoader
    >>> from DyslexiaInjector import DyslexiaInjector
    >>> dataset_wmt_enfr = load_dataset("wmt14",'fr-en', split='test')
    >>> to_translate = []
    >>> for i in range(len(dataset_wmt_enfr)):
    >>>     to_translate.append(dataset_wmt_enfr[i]['translation']['en'])
    >>> loader = DataLoader(data=to_translate, dataset_name="wmt14_enfr")
    >>> dyslexia_injector = DyslexiaInjector(loader)
    >>> dyslexia_injector.injection_swap(p_start=0.1, p_end=0.5, step_size=0.1, save_path="data/wmt14_enfr", save_format="both")
    This creates multiple files with different levels of dyslexia injected into the dataset. The files are saved in the data folder.
    """
    def __init__(self, load: DataLoader, 
                homophone_path = "data/homophones_dict.pickle",
                confusing_letters_path = "data/confusing_letters_dict.pickle",
                confusing_words_path="data/pedler_dict.pickle",
                seed = 42):
        self.load = load
        self.homophones_dict = self.load_dict(homophone_path)
        self.confusing_letters_dict = self.load_dict(confusing_letters_path)
        self.confusing_words_dict = self.load_dict(confusing_words_path)
        random.seed(seed)

    def load_dict(self, path):
        with open(path, "rb") as f:
            out = pickle.load(f)
            #close the file
            f.close()
        return out   

    def injection_swap(self, p_start=0, p_end=1, step_size=0.1, save_path="", save_format="both"):
        """
        Injects dyslexia into the dataset by swapping words and letters. It is to note, that probability p does not result in p% of the words being modified.
        For example, if p = 0.5, it does not mean that 50% of the words will be modified. It means that each word has a 50% chance of being modified. But, not all words
        have homophones, confusing letters or consufing words. Therefore, the actual percentage of words that are modified is lower than p. The same applies to letters.
        Parameters
        ----------
        p_start : float
            The starting probability of swapping a word and letter
        p_end : float
            The ending probability of swapping a word and letter
        step_size : float
            The step size of the probability
        save_path : str
            The path where the data needs to be saved
        save_format : str
            The format in which the data needs to be saved. Can be "both", "csv" or "txt"
        """
        df_swap_results = pd.DataFrame(columns=["dataset","p_homophone", "p_letter", "p_confusing_word", "homophones_injected",
                                        "letters_swapped", "confusing_words_injected", "words_modified", "sentences_changed"])
        #for loop that increases the p_homophone with step_size
        for i in np.arange(p_start, p_end+step_size, step_size):
            #round i to 3 decimals
            i = round(i, 3)
            for j in np.arange(p_start, p_end+step_size, step_size):
                #round j to 3 decimals
                j = round(j, 3)
                for k in np.arange(p_start, p_end+step_size, step_size):
                    k = round(k, 3)
                    #create deep copy of the data
                    temp_load, results = self.injection_runner(self.load.create_deepcopy(), i, j, k)
                    df_swap_results.loc[len(df_swap_results)] = {"dataset":"wmt14_en", "p_homophone":i, "p_letter":j, "p_confusing_word":k,
                                                                "homophones_injected":results[0],"letters_swapped":results[1], 
                                                                "confusing_words_injected": results[2], "words_modified":results[3], 
                                                                "sentences_changed":results[4]}
            return df_swap_results
        #efficiency wise this should be changed to lists and then combined at the end into a df
        df_swap_results = pd.DataFrame(columns=["filename","dataset","p_homophone", "p_letter", "p_confusing_word", "homophones_injected",
                                        "letters_swapped", "confusing_words_injected", "words_modified", "sentences_changed"])
        if individual:
            for i in range(3):
                p_homophone = 0
                p_letter = 0
                p_confusing_word = 0
                for k in np.arange(p_start+step_size, p_end+step_size, step_size):
                    k = round(k, 3)
                    if i == 0:
                        p_homophone = k
                    elif i == 1:
                        p_letter = k
                    elif i == 2:
                        p_confusing_word = k
                    df_swap_results = gather_save_results(self, df_swap_results, p_homophone, p_letter, p_confusing_word, save_path, save_format)
            df_swap_results = gather_save_results(self, df_swap_results, 0, 0, 0, save_path, save_format)
            
        else:
            #for loop that increases the p_homophone with step_size
            for i in np.arange(p_start, p_end+step_size, step_size):
                #round i to 3 decimals
                i = round(i, 3)
                for j in np.arange(p_start, p_end+step_size, step_size):
                    j = round(j, 3)
                    for k in np.arange(p_start, p_end+step_size, step_size):
                        k = round(k, 3)
                        #create deep copy of the data
                        df_swap_results = gather_save_results(self, df_swap_results, i, j, k, save_path, save_format)
                        
        #add number of sentences to the dataframe
        df_swap_results["sentences"] = self.load.get_number_of_sentences()
        #add number of words to the dataframe
        df_swap_results["words"] = self.load.get_number_of_words()
        #add number of characters to the dataframe
        df_swap_results["letters"] = self.load.get_number_of_letters()
        #percentage of sentences changed
        df_swap_results["percentage_sentences_changed"] = df_swap_results["sentences_changed"] / df_swap_results["sentences"] * 100
        #percentage of words modified
        df_swap_results["percentage_words_modified"] = df_swap_results["words_modified"] / df_swap_results["words"] * 100
        #percentaged of words swapped
        df_swap_results["percentage_words_swapped_for_homophones"] = df_swap_results["homophones_injected"] / df_swap_results["words"] * 100
        df_swap_results["percentage_words_swapped_for_confusing_words"] = df_swap_results["confusing_words_injected"] / df_swap_results["words"] * 100
        #percentage of letters swapped
        df_swap_results["percentage_letters_swapped"] = df_swap_results["letters_swapped"] / df_swap_results["letters"] * 100 
        #save the results
        df_swap_results.to_csv(f"{save_path}/swap_results.csv", index=False)
        return df_swap_results

    def injection_runner(self, data_loader, p_homophone, p_letter, p_confusing_word):
        #track of the amount of words that were swapped
        homophones_injected = 0
        #keep track of the amount of letters that were swapped
        letters_swapped = 0
        #keep track of the amount of confusing words that were injected
        confusing_words_injected = 0
        #track of the amount of words that were changed
        words_modified = 0
        #track of the amount of sentences that were changed
        sentences_changed = 0
        for i in range(len(data_loader.data)):
            #get the sentence
            sentence = data_loader.data[i]
            #swap the sentence
            sentence, results = self.injector(sentence, p_homophone, p_letter, p_confusing_word)
            # update the amount of words that were swapped
            homophones_injected += results[0]
            #update the amount of letters that were swapped
            letters_swapped += results[1]
            #updated the amount of confusing words that were injected
            confusing_words_injected += results[2]
            #update the amount of words that were changed
            words_modified += results[3]
            #update the amount of sentences that were changed
            if results[3] > 0:
                sentences_changed += 1
            #update the sentence in the dataframe
            data_loader.data[i] = sentence
        print(f"p_homophone = {p_homophone}, p_letter = {p_letter}, p_confusing_word = {p_confusing_word}")
        print("Homophones Injected: " + str(homophones_injected))
        print("Letters swapped: " + str(letters_swapped))
        print("Confusing words injected: " + str(confusing_words_injected))
        print("Words Modified: " + str(words_modified))
        print("Sentences changed: " + str(sentences_changed))
        return data_loader, (homophones_injected, letters_swapped, confusing_words_injected, words_modified, sentences_changed)

    def get_homophones(self, word):
        return self.homophones_dict[word]
    
    def get_homophone_dict(self):
        return self.homophones_dict
    
    def get_confusing_letters(self, word):
        return self.confusing_letters_dict[word]
    
    def get_confusing_letters_dict(self):
        return self.confusing_letters_dict
    
    def saver(self, temp_load: DataLoader, save_path, p_homophone, p_letter, p_confusing_word, format="both"):
        if format == "csv":
            temp_load.save_as_csv(save_path + f"{temp_load.get_name()}_p_homophone_{p_homophone}_p_letter_{p_letter}_p_confusing_word_{p_confusing_word}.csv")
        elif format == "txt":
            temp_load.save_as_txt(save_path + f"{temp_load.get_name()}_p_homophone_{p_homophone}_p_letter_{p_letter}_p_confusing_word_{p_confusing_word}.txt")
        else:
            #save as both
            temp_load.save_as_csv(save_path + f"{temp_load.get_name()}_p_homophone_{p_homophone}_p_letter_{p_letter}_p_confusing_word_{p_confusing_word}.csv")
            temp_load.save_as_txt(save_path + f"{temp_load.get_name()}_p_homophone_{p_homophone}_p_letter_{p_letter}_p_confusing_word_{p_confusing_word}.txt")

    def get_punctuation(self, word):
        #gets punctuationand symbols from a word
        punctuation = []
        for i in range(len(word)):
            if word[i] in '".,?!:;()' or word[i] == "'":
                punctuation.append((i,word[i]))
        return punctuation

    def homophone_swapper(self, in_word, out_word, apostrophe=False):
        #pick a random homophone from the list of homophones
        homophone = random.choice(self.homophones_dict[out_word])
        #check if difference is apostrophe
        if homophone.replace("'", "") == out_word:
            apostrophe = True
        #check if the first letter is capitalized
        if in_word.strip('".,?!:;()').strip("'")[0].isupper():
            #Capitalize the first letter of the homophone
            homophone = homophone[0].upper() + homophone[1:]
        #check if all the letters in the word are capitalized
        if in_word.isupper() and len(in_word) > 1:
            #capitalize the homophone
            homophone = homophone.upper()
        return homophone, apostrophe

    def confusing_word_injector(self, in_word, out_word):
        confusing_word = random.choice(self.confusing_words_dict[out_word])
        #check if the first letter is capitalized
        if in_word.strip('".,?!:;()').strip("'")[0].isupper():
            #Capitalize the first letter of the confusing word
            confusing_word = confusing_word[0].upper() + confusing_word[1:]
        #check if all the letters in the word are capitalized
        if in_word.isupper() and len(in_word) > 1:
            #capitalize the confusing word
            confusing_word = confusing_word.upper()
        return confusing_word

    def confusing_letter_swapper(self, in_word, out_word, p_letter, letters_swapped, homophone_swapped, confusing_word_swapped, confusing_letter_swapped):
        for i in range(len(out_word)):
                #check if swap a letter with a confusing letter with probability p_letter
                if  random.random() <= p_letter:
                    chance = random.random()
                    #need to skip first letter with 95% probability, based on Peddler findings
                    if i == 0 and chance >= 0.95:
                        continue
                    #check if the word is in the confusing letters dict
                    if out_word[i].lower() in self.confusing_letters_dict.keys():
                        #pick a random letter from the list of confusing letters
                        confusing_letter = random.choice(self.confusing_letters_dict[out_word[i].lower()])
                        #check if swapping a letter in a homophone
                        if not homophone_swapped and not confusing_word_swapped:
                            if in_word.strip('".,?!:;()').strip("'")[i].isupper():
                                confusing_letter = confusing_letter.upper()
                        else:
                            if out_word[i].isupper():
                                confusing_letter = confusing_letter.upper()
                        #replace the letter with the confusing letter at index j
                        out_word = out_word[:i] + confusing_letter + out_word[i+1:]
                        #update flag
                        confusing_letter_swapped = True
                        #update the amount of letters that were swapped
                        letters_swapped += 1
                        #lower the probability of swapping a letter with a confusing letter each time a letter is swapped
                        p_letter = 0.1*p_letter
                if not homophone_swapped and not confusing_word_swapped:
                    #ensure's proper capitalization of the word
                    if in_word.strip('".,?!:;()').strip("'")[i].isupper():
                        out_word = out_word[:i] + out_word[i].upper() + out_word[i+1:]
        return out_word, letters_swapped, confusing_letter_swapped


    def insert_punctuation(self, in_word, out_word, punctuation, apostrophe, homophone_swapped, confusing_word_swapped): #inwor car? outwor care
        if len(punctuation) == 0:
            return out_word
        for index, punc in punctuation:
                    if apostrophe and punc == "'" and index != 0 and index != len(in_word)-1:
                        continue
                    #if word already has that type of punctuation then skip it if its not at the first or last index
                    #also need to make sure its not punction that can be consecutive like "..." 
                    if index != 0 and index != len(in_word)-1 and punc in out_word:
                        if in_word[index-1] == punc and in_word[index+1] == punc:
                            out_word = out_word[:index] + punc + out_word[index:]
                        continue
                    if homophone_swapped or confusing_word_swapped:
                        if index == len(in_word)-1:
                            out_word = out_word + punc
                        continue
                    out_word = out_word[:index] + punc + out_word[index:]
        return out_word
    
    def injector(self, sentence, p_homophone, p_letter, p_confusing_word):
        #split the sentence into a list of words
        words = sentence.split()
        #keep track of the amount of homophones injected 
        homonphones_injected = 0
        #keep track of the amount of letters that were swapped
        letters_swapped = 0
        #keep track of the amount of confusing words injected
        confusing_words_injected = 0
        #keep track of the amount of words that were changed
        words_modified = 0
        for i in range(len(words)):
            #check for punctuation at all indexes of the word and save it
            punctuation = self.get_punctuation(words[i])         
            #get the word and remove any punctuation
            word = words[i].lower().strip('".,?!:;()')
            word = word.strip("'")
            #create a copy of the word to check if it was changed
            word_copy = words[i]
            #flag for homophone swapped and confusing letter swapped
            homophone_swapped = False
            confusing_letter_swapped = False
            confusing_word_swapped = False
            #flag for apostrophe
            apostrophe = False
            #check if the word is in the homophones dict
            if word in self.homophones_dict.keys():
                #check if the word has any homophones
                if len(self.homophones_dict[word]) > 0:
                    #swap the word with a homophone with probability p_homophone
                    if random.random() <= p_homophone:
                        #replace the word with the homophone, flag to see if apostrophe is the difference in homophone
                        word, apostrophe = self.homophone_swapper(words[i], word)
                        #update the amount of words that were swapped for homophone
                        homonphones_injected += 1
                        #update the flag
                        homophone_swapped = True
            #check if the word is in the pedler dict
            if word in self.confusing_words_dict.keys():
                #check if the word has any confusing words
                if len(self.confusing_words_dict[word]) > 0:
                    #swap the word with a homophone with probability p_homophone
                    if random.random() <= p_confusing_word:
                        word = self.confusing_word_injector(words[i], word)
                        #update the amount of words that were swapped for homophone
                        confusing_words_injected += 1
                        #update the flag
                        confusing_word_swapped = True
            #use confusing letter swapper to swap letters with probability p_letter
            word, letters_swapped, confusing_letter_swapped = self.confusing_letter_swapper(
                words[i], word, p_letter,
                letters_swapped, homophone_swapped,
                confusing_word_swapped, confusing_letter_swapped)
            #If whole word is upper case and its more than 1 letter then capitalize the whole word
            if words[i].isupper() and len(words[i]) > 1:
                word = word.upper()
            #replace the orignal word with new word and proper punctuation if any
            word = self.insert_punctuation(words[i], word, punctuation, apostrophe, homophone_swapped, confusing_word_swapped)
            if confusing_letter_swapped or homophone_swapped or confusing_word_swapped:
                words[i] = word
                words_modified += 1
        #join the list of words back into a sentence
        sentence = " ".join(words)
        return sentence, (homonphones_injected, letters_swapped, confusing_words_injected, words_modified)
            

In [13]:
class TestInjector(unittest.TestCase):
    
    def setUp(self):
        dataset_wmt_enfr = load_dataset("wmt14",'fr-en', split='test')
        to_translate_wmt14_en = []
        for i in range(len(dataset_wmt_enfr)):
            to_translate_wmt14_en.append(dataset_wmt_enfr[i]['translation']['en'])
        wmt14_en = DataLoader(data=to_translate_wmt14_en, dataset_name="wmt14_en")
        #The files paths for homophones and confusing letters will probably have to change
        self.injector =  DyslexiaInjector(load=wmt14_en, homophone_path="data/homophones_dict.pickle",
                                        confusing_letters_path="data/confusing_letters_dict.pickle", 
                                        confusing_words_path="data/pedler_dict.pickle", seed=3)
    
    def test_get_punctuation(self):
        test_string = "Hello-W.o.rld's?"
        expected_punctuation = [(7, '.'), (9, '.'), (13, "'"), (15, '?')]
        self.assertEqual(self.injector.get_punctuation(test_string), expected_punctuation)
    
    def test_homophone_swapper(self):
        original_word = "Capital,"
        word = "capital"
        apostrophe = False
        out_word, apostrophe = self.injector.homophone_swapper(original_word, word)
        #check that first letter is capitalized
        self.assertEqual(out_word[0].isupper(), True)
        #check that words are different
        self.assertNotEqual(out_word, word)
        self.assertEqual(out_word, "Capitol")
    
    def test_confusing_letter_swapper(self):
        homophone_swapped = False
        confusing_word_swapped = False
        confusing_letter_swapped = False
        letters_swapped = 0
        original_word = "Shh"
        word = "shh"
        out_word, letters_swapped, confusing_letter_swapped = self.injector.confusing_letter_swapper(
            original_word, word, 1, 
            letters_swapped, homophone_swapped, 
            confusing_word_swapped, 
            confusing_letter_swapped)
        if letters_swapped > 0:
            self.assertNotEqual(out_word, word)
        #check that word follows the same capitalization as the original word
        swapped_count = 0
        for i in range(len(out_word)):
            if original_word.strip('".,?!:;()').strip("'")[i].isupper():
                self.assertEqual(out_word[i].isupper(), True)
            else:
                self.assertEqual(out_word[i].islower(), True)
            if out_word[i].lower() != word[i].lower():
                swapped_count += 1
        #checking if correct number of letters were swapped
        self.assertEqual(swapped_count, letters_swapped, f"actual_swapped_count: {swapped_count} | given_letters_swapped: {letters_swapped}")
        #poissbile outcomes based on current version of confusing_letters_dict.pickle July 18 2023
        self.assertRegex(out_word, "Shn|Snn|Chh|Chn|Cnh|Cnn")

    def test_confusing_word_injector(self):
        original_word = "Back"
        word = "back"
        out_word = self.injector.confusing_word_injector(original_word, word)
        print(f"original word: {original_word} | out_word: {out_word}")
        self.assertNotEqual(out_word, word)

    def test_insert_punctuation(self):
        original_word = "ca,pital."
        word = "capital"
        punctuation = [(2, ","), (8, '.')]
        apostrophe = True
        out_word = self.injector.insert_punctuation(original_word, word, punctuation, apostrophe, False, False)
        self.assertEqual(out_word, original_word)

    def get_sentence_stats(self, original_sentence, new_sentence, k):
        actual_homophones = 0
        actual_words_modified = 0
        actual_letters_swapped = 0
        actual_confusing_words_injected = 0
        for i in range(len(original_sentence)):
            homophone_swapped = False
            confusing_word_swapped = False
            if original_sentence[i].lower().strip('".,?!:;()').strip("'") != new_sentence[i].lower().strip('".,?!:;()').strip("'"):
                actual_words_modified += 1
                #check if if new_sentence is a homophone of original_sentence
                if original_sentence[i].lower().strip('".,?!:;()').strip("'") in self.injector.homophones_dict and new_sentence[i].lower().strip('".,?!:;()').strip("'") in self.injector.homophones_dict[original_sentence[i].lower().strip('".,?!:;()').strip("'")] and k == 0:
                    actual_homophones += 1
                    homophone_swapped = True
                #check if confusing word was injected
                if original_sentence[i].lower().strip('".,?!:;()').strip("'") in self.injector.confusing_words_dict and new_sentence[i].lower().strip('".,?!:;()').strip("'") in self.injector.confusing_words_dict[original_sentence[i].lower().strip('".,?!:;()').strip("'")] and k == 2:
                    actual_confusing_words_injected += 1
                    confusing_word_swapped = True
                if not homophone_swapped and not confusing_word_swapped and k == 1:
                    #get the acutal number of letters swapped
                    for j in range(len(original_sentence[i].strip('".,?!:;()').strip("'"))):
                        if original_sentence[i][j].lower() != new_sentence[i][j].lower():
                            if new_sentence[i][j].lower().strip('".,?!:;()').strip("'") in self.injector.confusing_letters_dict[original_sentence[i][j].lower().strip('".,?!:;()').strip("'")]:
                                actual_letters_swapped += 1
        return actual_homophones, actual_words_modified, actual_letters_swapped, actual_confusing_words_injected               
                
    def test_injector(self):
        all_sentences= ['I am a sentence.', 'Also, in a sentence, we can have different types of punctuatioN!', "Now that we've tested punctuation, let's test homophones.","Let's see what happens when he have an abreviation like NASA"]
        #test once with p_homophone = 1 and p_letter = 0 and p_confusing_word = 0, then test with p_homophone = 0 and p_letter = 1 and p_confusing_word = 0, then test with p_homophone = 0 and p_letter = 0 and p_confusing_word = 1
        for original_sentence in all_sentences:
            for i in range(3):
                if i == 0:
                    out_sentence, results = self.injector.injector(original_sentence, p_homophone=1, p_letter=0, p_confusing_word=0)
                if i == 1:
                    out_sentence, results = self.injector.injector(original_sentence, p_homophone=0, p_letter=1, p_confusing_word=0)
                if i == 2:
                    out_sentence, results = self.injector.injector(original_sentence, p_homophone=0, p_letter=0, p_confusing_word=1)
                self.assertNotEqual(out_sentence, original_sentence)
                #also test to make sure the correct amount of words were changed and number of homophones injected
                expected_homophones = results[0]
                expected_letters_swapped= results[1]
                expected_confusing_words_injected = results[2]
                expected_words_modified= results[3]
                actual_homophones, actual_words_modified, actual_letters_swapped, actual_confusing_words_injected = self.get_sentence_stats(original_sentence.split(), out_sentence.split(), k=i)
                print(f"original_sentence: {original_sentence} | new_sentence: {out_sentence}")
                self.assertEqual(actual_homophones, expected_homophones, f"actual_homophones: {actual_homophones} | expected_homophones: {expected_homophones}, i: {i}")
                self.assertEqual(actual_words_modified, expected_words_modified, f"actual_words_modified: {actual_words_modified} | expected_words_modified: {expected_words_modified}, i: {i}")
                self.assertEqual(actual_letters_swapped, expected_letters_swapped, f"actual_letters_swapped: {actual_letters_swapped} | expected_letters_swapped: {expected_letters_swapped}, i: {i}")
                self.assertEqual(actual_confusing_words_injected, expected_confusing_words_injected, f"actual_confusing_words_injected: {actual_confusing_words_injected} | expected_confusing_words_injected: {expected_confusing_words_injected}, i: {i}")


In [14]:
from datasets import load_dataset
dataset_wmt_enfr = load_dataset("wmt14",'fr-en', split='test')

Found cached dataset wmt14 (C:/Users/gerge/.cache/huggingface/datasets/wmt14/fr-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)


In [15]:
to_translate_wmt14_en = []
reference_wmt14_fr = []
for i in range(len(dataset_wmt_enfr)):
    to_translate_wmt14_en.append(dataset_wmt_enfr[i]['translation']['en'])
    reference_wmt14_fr.append(dataset_wmt_enfr[i]['translation']['fr'])

In [16]:
to_translate_wmt14_en

['Spectacular Wingsuit Jump Over Bogota',
 'Sportsman Jhonathan Florez jumped from a helicopter above Bogota, the capital of Colombia, on Thursday.',
 'Wearing a wingsuit, he flew past over the famous Monserrate Sanctuary at 160km/h. The sanctuary is located at an altitude of over 3000 meters and numerous spectators had gathered there to watch his exploit.',
 'A black box in your car?',
 "As America's road planners struggle to find the cash to mend a crumbling highway system, many are beginning to see a solution in a little black box that fits neatly by the dashboard of your car.",
 "The devices, which track every mile a motorist drives and transmit that information to bureaucrats, are at the center of a controversial attempt in Washington and state planning offices to overhaul the outdated system for funding America's major roads.",
 'The usually dull arena of highway planning has suddenly spawned intense debate and colorful alliances.',
 'Libertarians have joined environmental groups

In [17]:
wmt14_en = DataLoader(data=to_translate_wmt14_en, dataset_name="wmt14_en")

In [18]:
class TestSentence(unittest.TestCase):
    def setUp(self):
        self.data = wmt14_en.get_data() #prob need to change this
    #this is unicode characters that sometimes slip through
    def test_for_xa0(self):
        for sentence in self.data:
            self.assertNotIn('xa0', sentence, sentence)
    def test_for_french_quotes(self):
        #testing related to french dataset
        for sentence in self.data:
            self.assertNotIn("«", sentence, sentence)
            self.assertNotIn("»", sentence, sentence)
    def test_for_newline(self):
        for sentence in self.data:
            self.assertNotIn("\n", sentence, sentence)
    def test_for_backslash(self):
        for sentence in self.data:
            self.assertNotIn("\\", sentence, sentence)
    def test_for_german_quotes(self):
        for sentence in self.data:
            self.assertNotIn("„", sentence, sentence)
            self.assertNotIn("“", sentence, sentence)
    def test_for_quote_number(self):
        for sentence in self.data:
            #make sure there is 0 or an even number of quotes
            self.assertEqual(sentence.count('"') % 2, 0, sentence)


In [19]:
#run tests
unittest.main(argv=[''], verbosity=2, exit=False)

test_confusing_letter_swapper (__main__.TestInjector) ... Found cached dataset wmt14 (C:/Users/gerge/.cache/huggingface/datasets/wmt14/fr-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)
ok
test_confusing_word_injector (__main__.TestInjector) ... Found cached dataset wmt14 (C:/Users/gerge/.cache/huggingface/datasets/wmt14/fr-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)
ok
test_get_punctuation (__main__.TestInjector) ... 

original word: Back | out_word: Beck


Found cached dataset wmt14 (C:/Users/gerge/.cache/huggingface/datasets/wmt14/fr-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)
ok
test_homophone_swapper (__main__.TestInjector) ... Found cached dataset wmt14 (C:/Users/gerge/.cache/huggingface/datasets/wmt14/fr-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)
ok
test_injector (__main__.TestInjector) ... Found cached dataset wmt14 (C:/Users/gerge/.cache/huggingface/datasets/wmt14/fr-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)
ok
test_insert_punctuation (__main__.TestInjector) ... 

original_sentence: I am a sentence. | new_sentence: I am uh sentence.
original_sentence: I am a sentence. | new_sentence: E im u centence.
original_sentence: I am a sentence. | new_sentence: I an he sentence.
original_sentence: Also, in a sentence, we can have different types of punctuatioN! | new_sentence: Also, inn uh sentence, wee can halve different types of punctuatioN!
original_sentence: Also, in a sentence, we can have different types of punctuatioN! | new_sentence: Olco, un e centence, me ean nave pifferent fypec ef qunctiatioN!
original_sentence: Also, in a sentence, we can have different types of punctuatioN! | new_sentence: Also, inn eh sentence, wee canoe hive difference types off punctuatioN!
original_sentence: Now that we've tested punctuation, let's test homophones. | new_sentence: Now that weave tested punctuation, lets test homophones.
original_sentence: Now that we've tested punctuation, let's test homophones. | new_sentence: How tnat me've fested qunctuation, iet's f

Found cached dataset wmt14 (C:/Users/gerge/.cache/huggingface/datasets/wmt14/fr-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)
ok
test_for_backslash (__main__.TestSentence) ... ok
test_for_french_quotes (__main__.TestSentence) ... ok
test_for_german_quotes (__main__.TestSentence) ... ok
test_for_newline (__main__.TestSentence) ... ok
test_for_quote_number (__main__.TestSentence) ... FAIL
test_for_xa0 (__main__.TestSentence) ... ok

FAIL: test_for_quote_number (__main__.TestSentence)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\gerge\AppData\Local\Temp\ipykernel_9268\163673438.py", line 28, in test_for_quote_number
    self.assertEqual(sentence.count('"') % 2, 0, sentence)
AssertionError: 1 != 0 : or sell it to someone else, which is what happened," explains Paul Arsenault, holder of the Transat Chair in Tourism at the School of Management at the UQAM.

-----------------------------

In [20]:
data_injector_wmt14_en = DyslexiaInjector(load=wmt14_en, homophone_path="data/homophones_dict.pickle", confusing_letters_path="data/confusing_letters_dict.pickle", confusing_words_path="data/pedler_dict.pickle")

In [22]:
data_injector_wmt14_en.injection_swap(p_start=0, p_end=0.05, step_size=0.025, save_path="to_test/", individual=True)

p_homophone = 0.025, p_letter = 0, p_confusing_word = 0
Homophones Injected: 436
Letters swapped: 0
Confusing words injected: 0
Words Modified: 436
Sentences changed: 398
Saved wmt14_en to to_test/wmt14_en_p_homophone_0.025_p_letter_0_p_confusing_word_0.csv
Saved wmt14_en to to_test/wmt14_en_p_homophone_0.025_p_letter_0_p_confusing_word_0.txt
Saved wmt14_en to to_test/wmt14_en_p_homophone_0.025_p_letter_0_p_confusing_word_0.docx
p_homophone = 0.05, p_letter = 0, p_confusing_word = 0
Homophones Injected: 889
Letters swapped: 0
Confusing words injected: 0
Words Modified: 889
Sentences changed: 730
Saved wmt14_en to to_test/wmt14_en_p_homophone_0.05_p_letter_0_p_confusing_word_0.csv
Saved wmt14_en to to_test/wmt14_en_p_homophone_0.05_p_letter_0_p_confusing_word_0.txt
Saved wmt14_en to to_test/wmt14_en_p_homophone_0.05_p_letter_0_p_confusing_word_0.docx
p_homophone = 0.075, p_letter = 0, p_confusing_word = 0
Homophones Injected: 1436
Letters swapped: 0
Confusing words injected: 0
Words Mod

filename   dataset  p_homophone   
0  wmt14_en_p_homophone_0.025_p_letter_0_p_confus...  wmt14_en        0.025  \
1  wmt14_en_p_homophone_0.05_p_letter_0_p_confusi...  wmt14_en        0.050   
2  wmt14_en_p_homophone_0.075_p_letter_0_p_confus...  wmt14_en        0.075   
3  wmt14_en_p_homophone_0_p_letter_0.025_p_confus...  wmt14_en        0.000   
4  wmt14_en_p_homophone_0_p_letter_0.05_p_confusi...  wmt14_en        0.000   
5  wmt14_en_p_homophone_0_p_letter_0.075_p_confus...  wmt14_en        0.000   
6  wmt14_en_p_homophone_0_p_letter_0_p_confusing_...  wmt14_en        0.000   
7  wmt14_en_p_homophone_0_p_letter_0_p_confusing_...  wmt14_en        0.000   
8  wmt14_en_p_homophone_0_p_letter_0_p_confusing_...  wmt14_en        0.000   
9  wmt14_en_p_homophone_0_p_letter_0_p_confusing_...  wmt14_en        0.000   

   p_letter  p_confusing_word  homophones_injected  letters_swapped   
0     0.000             0.000                  436                0  \
1     0.000             0.000                  889                0   
2     0.000             0.000                 1436                0   
3     0.025             0.000                    0             6994   
4     0.050             0.000                    0            12994   
5     0.075             0.000                    0            18323   
6     0.000             0.025                    0                0   
7     0.000             0.050                    0                0   
8     0.000             0.075                    0                0   
9     0.000             0.000                    0                0   

   confusing_words_injected  words_modified  sentences_changed  sentences   
0                         0             436                398       3003  \
1                         0             889                730       3003   
2                         0            1436               1086       3003   
3                         0            6950               2538       3003   
4                         0           12837               2852       3003   
5                         0           17961               2943       3003   
6                       891             891                747       3003   
7                      1777            1777               1272       3003   
8                      2862            2862               1726       3003   
9                         0               0                  0       3003   

   words  letters  percentage_sentences_changed  percentage_words_modified   
0  62336   362045                     13.253413                   0.699435  \
1  62336   362045                     24.309024                   1.426142   
2  62336   362045                     36.163836                   2.303645   
3  62336   362045                     84.515485                  11.149256   
4  62336   362045                     94.971695                  20.593237   
5  62336   362045                     98.001998                  28.813206   
6  62336   362045                     24.875125                   1.429351   
7  62336   362045                     42.357642                   2.850680   
8  62336   362045                     57.475857                   4.591247   
9  62336   362045                      0.000000                   0.000000   

   percentage_words_swapped_for_homophones   
0                                 0.699435  \
1                                 1.426142   
2                                 2.303645   
3                                 0.000000   
4                                 0.000000   
5                                 0.000000   
6                                 0.000000   
7                                 0.000000   
8                                 0.000000   
9                                 0.000000   

   percentage_words_swapped_for_confusing_words  percentage_letters_swapped  
0                                      0.000000                    0.000000  
1                     